In [4]:
import time
import keras

from keras.models import Model
from keras.layers import Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Add, Concatenate
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, TimeDistributed, Reshape
import keras.backend as K
import keras.callbacks as callbacks

import pandas as pd
import numpy as np
from numpy import array
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

2023-04-07 10:33:19.855259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:

save_path = "/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/"
model_name = "conv_only"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 10

#### define functions ####

def model_cycle():
    inputs = Input(shape=(50, 4, 1))
        
    x = Conv2D(48, kernel_size=(3,4),
                   activation='relu',
                   padding='valid')(inputs)
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # print(f"After first convolutional layer: x.shape = {x.shape}")

    # x = Reshape((K.int_shape(x)[1], K.int_shape(x)[3]))(x)
    # x = Conv1D(48, kernel_size=(11),
    #                activation='relu',
    #                padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # print(f"After second convolutional layer: x.shape = {x.shape}")

    # x = Conv2D(48, kernel_size=(21,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = MaxPooling2D((24,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # print(f"After third convolutional layer, x.shape = {x.shape}")

    # x = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)
    # x = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = MaxPooling2D((12,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # # parallel line 1
    # fx1 = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # fx1 = BatchNormalization()(fx1)
    # fx1 = Dropout(0.2)(fx1)
    # fx1 = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(fx1)
    # fx1 = MaxPooling2D((2,1),padding='same')(fx1)
    # fx1 = BatchNormalization()(fx1)
    # fx1 = Dropout(0.2)(fx1)
    
    # # parallel line 2
    # fx2 = Conv2D(48, kernel_size=(11,1),
    #                activation='relu',
    #                padding='same')(x)
    # fx2 = BatchNormalization()(fx2)
    # fx2 = Dropout(0.2)(fx2)
    # fx2 = Conv2D(48, kernel_size=(21,1),
    #                activation='relu',
    #                padding='same')(fx2)
    # fx2 = MaxPooling2D((2,1),padding='same')(fx2)
    # fx2 = BatchNormalization()(fx2)
    # fx2 = Dropout(0.2)(fx2)
    
    # # # Add
    # x1 = Concatenate(axis=-3)([fx1, fx2])
    # x = Add()([x, x1])
    # x = MaxPooling2D((2,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)
    
    # x = Reshape((K.int_shape(x)[1], K.int_shape(x)[3]))(x)
    # x = LSTM(20, return_sequences=False)(x)
    # x = Dropout(0.2)(x)

    # x = Reshape((1, 24*48))(x)
    x = Flatten()(x)

    outputs = Dense(1, activation='linear')(x)
    # print(outputs.shape)
    network = Model(inputs, outputs)
    network.compile(optimizer='rmsprop',
                    loss='mean_squared_error')
    return network
    
def dnaOneHot(sequence):
    seq_array = array(list(sequence))
    code = {"A": [0], "C": [1], "G": [2], "T": [3], "N": [4],
            "a": [0], "c": [1], "g": [2], "t": [3], "n": [4]}
    onehot_encoded_seq = []
    for char in seq_array:
        onehot_encoded = np.zeros(5)
        onehot_encoded[code[char]] = 1
        onehot_encoded_seq.append(onehot_encoded[0:4])
    return onehot_encoded_seq



In [1]:
def display_fits(fits):
    print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 5])}",
          f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 5])}",
          f"\nAverage correlation on random: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 3])}",
          f"\nAverage MSE on random: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 3])}",
          f"\nAverage correlation on ChrV: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 7])}",
          f"\nAverage MSE on ChrV: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 7])}",
          f"\nAverage correlation on CN: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 0])}",
          f"\nAverage MSE on CN: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 0])}")

In [4]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], np.transpose(inv_mat)))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [5]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.process_time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.process_time() - self.epoch_time_start)
        
#### preparing data ####

data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle1.txt",delimiter = ",")
X1 = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1.append(dnaOneHot(sequence_nt))
X1 = array(X1)
X1 = X1.reshape((X1.shape[0],50,4,1))
X1_reverse = np.flip(X1,[1,2])
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)
Y1 = find_c0new(data_cerevisiae_nucle).astype(float)


In [6]:
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle3.txt",delimiter = ",")
X3 = []
for sequence_nt in data_random_library["Sequence"]:
    X3.append(dnaOneHot(sequence_nt))
X3 = array(X3)
X3 = X3.reshape((X3.shape[0],50,4,1))
X3_reverse = np.flip(X3,[1,2])
# Y3 = data_random_library["C0"].values.astype(float)
Y3 = find_c0new(data_random_library).astype(float)


In [7]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(dnaOneHot(sequence_nt))
X5 = array(X5)
X5 = X5.reshape((X5.shape[0],50,4,1))
X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = find_c0new(data_tiling).astype(float)



In [8]:
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle6.txt",delimiter = ",")
X6 = []
for sequence_nt in data_chr5["Sequence"]:
    X6.append(dnaOneHot(sequence_nt))
X6 = array(X6)
X6 = X6.reshape((X6.shape[0],50,4,1))
X6_reverse = np.flip(X6,[1,2])
# Y6 = data_chr5["C0"].values.astype(float)
Y6 = find_c0new(data_chr5).astype(float)

In [9]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
n = Y5.shape[0]

fits = []
detrend = []
times = []
times2 = []

for train_index, val_index in kf.split(Y5):
    training_X = X5[train_index]
    training_X_reverse = X5_reverse[train_index]
    validation_X = X5[val_index]
    validation_X_reverse = X5_reverse[val_index]
    training_Y = Y5[train_index]
    validation_Y = Y5[val_index]
    # CREATE NEW MODEL
    model = model_cycle()
    # CREATE CALLBACKS
    checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_tiling_"+str(fold_var)+".h5",
                                                    monitor='val_loss', verbose=1,
                                                    save_best_only=True, mode='min')
    time_callback = TimeHistory()

    history = model.fit(training_X, training_Y,
                        epochs=num_epochs,
                        callbacks= [checkpoint, time_callback],
                        validation_data=(validation_X, validation_Y))
    model.load_weights(save_path + model_name+"_tiling_"+str(fold_var)+".h5")
    model.save(save_path+model_name+"_tiling_"+str(fold_var),save_traces=False)
    times.append(time_callback.times)

    pred_Y = model.predict(training_X)
    pred_Y = pred_Y.reshape(pred_Y.shape[0])
    pred_Y_reverse = model.predict(training_X_reverse)
    pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
    pred_Y = (pred_Y+pred_Y_reverse)/2
    reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    
    detrend_int = reg.intercept_
    detrend_slope = reg.coef_
    detrend.append([float(detrend_int), float(detrend_slope)])

    start_time = time.process_time()
    fit = model.predict(X1)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X1_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X3)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X3_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(validation_X)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(validation_X_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X6)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X6_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    K.clear_session()
    fold_var += 1
    
detrend = array(detrend)
detrend = pd.DataFrame(detrend)
detrend.to_csv(save_path +model_name+"_detrend_tiling.txt", index = False)

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+ "_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')

with open(save_path +model_name+"_pred_time_tiling.txt", "w") as file:
    for row in times2:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch 1/10


2023-04-06 17:29:32.542413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2295/2317 [============================>.] - ETA: 0s - loss: 0.2819
Epoch 1: val_loss improved from inf to 0.21694, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tiling_1.h5
2317/2317 [==============================] - 5s 2ms/step - loss: 0.2814 - val_loss: 0.2169
Epoch 2/10
2310/2317 [============================>.] - ETA: 0s - loss: 0.2188
Epoch 2: val_loss did not improve from 0.21694
2317/2317 [==============================] - 5s 2ms/step - loss: 0.2187 - val_loss: 0.2192
Epoch 3/10
2312/2317 [============================>.] - ETA: 0s - loss: 0.2175
Epoch 3: val_loss improved from 0.21694 to 0.21272, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tiling_1.h5
2317/2317 [==============================] - 6s 3ms/step - loss: 0.2176 - val_loss: 0.2127
Epoch 4/10
2303/2317 [============================>.] - ETA: 0s - loss: 0.2171
Epoch 4: val_loss did not improve from 

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tiling_1/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tiling_1/assets


...

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tiling_9/assets


2576/2576 [==============================] - 2s 618us/step
Epoch 1/10
2287/2317 [============================>.] - ETA: 0s - loss: 0.2808
Epoch 1: val_loss improved from inf to 0.21682, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tiling_10.h5
2317/2317 [==============================] - 4s 2ms/step - loss: 0.2799 - val_loss: 0.2168
Epoch 2/10
2298/2317 [============================>.] - ETA: 0s - loss: 0.2173
Epoch 2: val_loss did not improve from 0.21682
2317/2317 [==============================] - 4s 2ms/step - loss: 0.2173 - val_loss: 0.2210
Epoch 3/10
2311/2317 [============================>.] - ETA: 0s - loss: 0.2159
Epoch 3: val_loss did not improve from 0.21682
2317/2317 [==============================] - 4s 2ms/step - loss: 0.2159 - val_loss: 0.2188
Epoch 4/10
2287/2317 [============================>.] - ETA: 0s - loss: 0.2149
Epoch 4: val_loss improved from 0.21682 to 0.21339, saving model to /Users/Brody1/Documents

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tiling_10/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tiling_10/assets


2576/2576 [==============================] - 2s 715us/step


In [33]:
fits = pd.read_csv(save_path +model_name+ "_fits_tiling.txt",delimiter = ",")
fits=array(fits.values.tolist())
fits = np.transpose(fits)

In [34]:
display_fits(fits)

Average correlation on tiling: 0.3552491751584278 
Average MSE on tiling: 0.2116396255226801 
Average correlation on random: 0.39398630477831575 
Average MSE on random: 0.12187173926202433 
Average correlation on ChrV: 0.2831865412110821 
Average MSE on ChrV: 0.2676530816022441 
Average correlation on CN: 0.2967467853894225 
Average MSE on CN: 0.1980794400475368


In [11]:
model = keras.models.load_model(save_path + model_name + "_tiling_10")

In [12]:
first_conv_model = Model(inputs = model.input, outputs = model.layers[1].output)
first_conv_output = first_conv_model.predict(X5)
pd.DataFrame(first_conv_output.reshape(first_conv_output.shape[0], -1)).to_csv(save_path + model_name+"_tiling_tiling_first_conv_output")
first_conv_output_random = first_conv_model.predict(X3)
pd.DataFrame(first_conv_output_random.reshape(first_conv_output_random.shape[0], -1)).to_csv(save_path + model_name+"_tiling_random_first_conv_output")


390/390 [==============================] - 0s 600us/step


In [13]:
first_conv_output[0][47]

array([[0.        , 0.37548894, 0.        , 0.        , 0.        ,
        0.63033015, 0.17333601, 0.        , 0.        , 0.4276547 ,
        0.        , 0.        , 0.        , 0.        , 0.35167992,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.5217654 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.16024733,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.3988306 , 0.        , 0.        ,
        0.        , 0.        , 0.        ]], dtype=float32)

In [14]:
first_conv_weights = model.layers[1].weights[0]
first_conv_biases = model.layers[1].weights[1]
# Format: [Position 0: A, C, G, T, Position 1: A, C, G, T, Position 2: A, C, G, T]
pd.DataFrame(array(first_conv_weights).transpose((3,2,0,1)).reshape(first_conv_weights.shape[-1], -1)).to_csv(save_path + model_name+"_tiling_first_conv_kernels")
pd.DataFrame(first_conv_biases).to_csv(save_path + model_name+"_tiling_first_conv_biases")

In [15]:
print(first_conv_weights)

<tf.Variable 'conv2d/kernel:0' shape=(3, 4, 1, 48) dtype=float32, numpy=
array([[[[-8.15694332e-02,  1.06328567e-02, -4.22265008e-02,
          -9.87932738e-03, -8.39487538e-02,  4.30427603e-02,
          -1.67828426e-01,  2.96451226e-02, -5.20521179e-02,
           3.25803980e-02, -7.31743947e-02, -5.79910539e-02,
           3.58043052e-02, -7.67797530e-02, -9.07838494e-02,
          -7.02487975e-02,  6.50768578e-02,  1.58567369e-01,
           4.29326203e-03, -3.68713252e-02, -3.29190157e-02,
          -2.40225643e-01,  2.68410370e-02, -3.01064327e-02,
          -7.29377717e-02, -2.09992863e-02, -2.22886577e-02,
          -2.20489055e-02, -2.45698895e-02, -3.05987913e-02,
           2.00851075e-02, -5.59120020e-03,  6.01138920e-02,
          -4.43689078e-02, -6.31678775e-02, -4.54354808e-02,
          -1.32699031e-02, -8.74557421e-02, -1.09437108e-01,
           2.10127910e-03, -9.16371942e-02, -2.67568454e-02,
           2.74200849e-02, -1.03594493e-02,  2.00665519e-02,
          -7

In [16]:
first_conv_output[0][0][0][10]

0.0

In [17]:
print(X5.shape)

(82368, 50, 4, 1)


In [18]:
[X5[0][i:i+3] for i in range(48)]*array(first_conv_weights)

ValueError: operands could not be broadcast together with shapes (48,3,4,1) (3,4,1,48) 

In [19]:
print(array(first_conv_weights))

[[[[-8.15694332e-02  1.06328567e-02 -4.22265008e-02 -9.87932738e-03
    -8.39487538e-02  4.30427603e-02 -1.67828426e-01  2.96451226e-02
    -5.20521179e-02  3.25803980e-02 -7.31743947e-02 -5.79910539e-02
     3.58043052e-02 -7.67797530e-02 -9.07838494e-02 -7.02487975e-02
     6.50768578e-02  1.58567369e-01  4.29326203e-03 -3.68713252e-02
    -3.29190157e-02 -2.40225643e-01  2.68410370e-02 -3.01064327e-02
    -7.29377717e-02 -2.09992863e-02 -2.22886577e-02 -2.20489055e-02
    -2.45698895e-02 -3.05987913e-02  2.00851075e-02 -5.59120020e-03
     6.01138920e-02 -4.43689078e-02 -6.31678775e-02 -4.54354808e-02
    -1.32699031e-02 -8.74557421e-02 -1.09437108e-01  2.10127910e-03
    -9.16371942e-02 -2.67568454e-02  2.74200849e-02 -1.03594493e-02
     2.00665519e-02 -7.62655661e-02 -1.08363386e-03 -1.77751742e-02]]

  [[-1.48135787e-02  1.59049630e-01 -1.48712425e-02  6.45468235e-02
     1.09931901e-01  7.77474642e-02  6.18786812e-02  4.31310944e-02
    -2.36688536e-02  8.44759718e-02 -9.514352

In [20]:
# first_conv_output = first_conv_model.predict(X5)
print(first_conv_output[0][0][0])




[0.         0.51478916 0.         0.         0.         0.48996627
 0.25058284 0.         0.         0.3336922  0.         0.
 0.         0.         0.19565895 0.         0.         0.35314748
 0.         0.         0.         0.49628997 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.61808795 0.         0.         0.         0.         0.        ]


In [21]:
print(array(model.layers[1].weights[0][0][0][0][0]))
print(model.layers[1].weights[0][0][1][0][0])

print([array(weights[0][0]) for weights in model.layers[1].weights[0][0]])
print([array(weights[0][0]) for weights in model.layers[1].weights[0][1]])


print([array(weights[0][0]).item() for subweights in model.layers[1].weights[0] for weights in subweights])
print([array(weights[0][1]).item() for subweights in model.layers[1].weights[0] for weights in subweights])


print([[array(weights[0][i]).item()for subweights in model.layers[1].weights[0] for weights in subweights] for i in range(48)])



# print(pd.DataFrame(model.layers[1].weights.reshape(3,4,-1)))

-0.08156943
tf.Tensor(-0.014813579, shape=(), dtype=float32)
[array(-0.08156943, dtype=float32), array(-0.01481358, dtype=float32), array(-0.02392906, dtype=float32), array(-0.00820983, dtype=float32)]
[array(-0.05773938, dtype=float32), array(-0.0174441, dtype=float32), array(-0.02526142, dtype=float32), array(-0.01325114, dtype=float32)]
[-0.08156943321228027, -0.014813578687608242, -0.023929063230752945, -0.008209830150008202, -0.05773937702178955, -0.017444096505641937, -0.025261420756578445, -0.013251136988401413, 0.057782236486673355, -0.00603526271879673, -0.06302516907453537, -0.09946046769618988]
[0.010632856748998165, 0.1590496301651001, 0.1674755960702896, -0.307123064994812, 0.009844825603067875, 0.1011139526963234, -0.05654657632112503, 0.06222948804497719, 0.09942752867937088, 0.13365577161312103, 0.11152572929859161, 0.05741782858967781]
[[-0.08156943321228027, -0.014813578687608242, -0.023929063230752945, -0.008209830150008202, -0.05773937702178955, -0.01744409650564193

In [22]:
print(model.layers[1].weights[0][1])

tf.Tensor(
[[[-0.05773938  0.00984483  0.01871413 -0.04371226 -0.07880948
   -0.12998372  0.13833761 -0.07962317  0.0180073  -0.3124479
    0.00583496 -0.0746757   0.04148425 -0.04732831  0.02001379
   -0.09126399  0.00608644  0.06725612 -0.08626434 -0.00723102
    0.03870613  0.03016573 -0.10017101 -0.02541168  0.00896178
    0.00678906  0.03268983 -0.06514793 -0.04381192 -0.06193892
   -0.0576072  -0.06167997 -0.05143496 -0.03384411 -0.07685674
    0.03579092 -0.09214066 -0.07506205  0.12282153 -0.05130965
    0.03287876 -0.01106898 -0.09437226 -0.08237112 -0.01837485
   -0.10669442  0.00091816 -0.07953299]]

 [[-0.0174441   0.10111395 -0.00855389 -0.01962516  0.11107911
    0.15719867  0.0768749  -0.00439545  0.00986179  0.21125792
   -0.05541576 -0.07026222  0.04599626  0.05640414  0.18894874
    0.07040861 -0.09760433  0.06239343 -0.01761479 -0.05667232
    0.00102776 -0.10166962 -0.0054029   0.00554748  0.00928201
   -0.00377133 -0.01785539 -0.00535342  0.03546768 -0.11950861
   

In [23]:
print(X5[0])

[[[0.]
  [0.]
  [1.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [1.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [1.]
  [0.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [1.]
  [0.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [1.]
  [0.]]

 [[0.]
  [0.]
  [1.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [0.]
  [1.]
  [0.]]

 [[1.]
  [

In [24]:
first_conv_weights = model.layers[1].weights[0]
first_conv_biases = model.layers[1].weights[1]
# Format: [Position 0: A, C, G, T, Position 1: A, C, G, T, Position 2: A, C, G, T]
pd.DataFrame(array(first_conv_weights).transpose((3,2,0,1)).reshape(first_conv_weights.shape[-1], -1)).to_csv(save_path + model_name+"_tiling_first_conv_kernels")
pd.DataFrame(first_conv_biases).to_csv(save_path + model_name+"_tiling_first_conv_biases")